<a href="https://colab.research.google.com/github/kooll/ThinkPythonJ/blob/main/chapters/chap17_translated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Think Python 3e* の印刷版および電子書籍版は、[Bookshop.org](https://bookshop.org/a/98697/9781098155438)と[Amazon](https://www.amazon.com/_/dp/1098155432?smid=ATVPDKIKX0DER&_encoding=UTF8&tag=oreilly20-20&_encoding=UTF8&tag=greenteapre01-20&linkCode=ur2&linkId=e2a529f94920295d27ec8a06e757dc7c&camp=1789&creative=9325)から注文できます。

In [ ]:
from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + str(local))
    return filename

download('https://github.com/AllenDowney/ThinkPython/raw/v3/thinkpython.py');
download('https://github.com/AllenDowney/ThinkPython/raw/v3/diagram.py');

import thinkpython

# 継承

オブジェクト指向プログラミングと最も関連付けられる言語機能は、**継承**です。継承は、既存のクラスを元にして、それを修正した新しいクラスを定義する機能です。この章では、トランプのカード、カードのデッキ、ポーカーの手役を表すクラスを使って継承を示します。もしポーカーをプレイしないとしても心配しないでください ― 必要なことはちゃんと説明します。

## カードの表現

標準的なデッキには52枚のトランプカードがあり、それぞれは4つのスーツのうちの1つに属し、13のランクのうちの1つに属します。
スーツはスペード、ハート、ダイヤモンド、クラブです。
ランクはエース、2、3、4、5、6、7、8、9、10、ジャック、クイーン、キングです。
プレイするゲームによっては、エースがキングより高いか、2より低いかを決定します。

トランプカードを表す新しいオブジェクトを定義したい場合、属性は `rank` と `suit` であることは明らかです。
しかし、属性の型を何にするかはあまり明らかではありません。
一つの可能性は、スーツには `'Spade'` のような文字列を、ランクには `'Queen'` のような文字列を使用することです。
この実装の問題点は、カードのランクやスーツを比較してどちらが高いかを判断するのが容易ではないことです。

代替案として、ランクとスーツをエンコードするために整数を使用する方法があります。
この文脈で「エンコード」とは、数値とスーツ、または数値とランクの間にマッピングを定義することを意味します。
この種のエンコードは秘密にすることが目的ではありません（それは「暗号化」となります）。

例えば、この表はスーツとそれに対応する整数コードを示しています。

| スーツ | コード |
| --- | --- |
| スペード | 3 |
| ハート | 2 |
| ダイヤ | 1 |
| クラブ | 0 |

このエンコーディングを使用すると、スーツをそのコードを比較することで比較することができます。

ランクをエンコードするために、整数 `2` をランク `2`、`3` を `3`、そして `10` まで順に使用します。フェイスカードに対するコードは次の表の通りです。

| ランク | コード |
| --- | --- |
| ジャック | 11 |
| クイーン | 12 |
| キング | 13 |

エースを表すには、他のランクより低く扱う場合は `1`、高く扱う場合は `14` を使用します。

これらのエンコーディングを表現するために、スートの名前を表す文字列のリストと、ランクの名前を表す文字列のリストを使用します。

以下は、トランプのカードを表すクラスの定義です。このクラスには**クラス変数**があり、これはクラス定義内で定義される変数ですが、メソッド内ではありません。

In [ ]:
class Card:
    """Represents a standard playing card."""

    suit_names = ['Clubs', 'Diamonds', 'Hearts', 'Spades']
    rank_names = [None, 'Ace', '2', '3', '4', '5', '6', '7',
                  '8', '9', '10', 'Jack', 'Queen', 'King', 'Ace']

`rank_names` の最初の要素が `None` であるのは、ランクがゼロのカードが存在しないためです。この `None` をプレースキーパーとして含めることにより、インデックス `2` が文字列 `'2'` にマップするという便利な特性を持つリストを得られます。

クラス変数はクラスに関連付けられており、そのクラスのインスタンスではなく、次のようにアクセスできます。

In [ ]:
Card.suit_names

`suit_names`を使用してスーツを調べ、その対応する文字列を取得できます。

In [ ]:
Card.suit_names[0]

「rank_names」を使用してランクを検索します。

In [ ]:
Card.rank_names[11]

## カードの属性

こちらは `Card` クラスの `__init__` メソッドです。このメソッドは `suit` と `rank` をパラメータとして取り、それらを同じ名前の属性に割り当てます。

In [ ]:
%%add_method_to Card

    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank

これで、このようにして `Card` オブジェクトを作成できます。

In [ ]:
queen = Card(1, 12)

新しいインスタンスを使用して属性にアクセスできます。

In [ ]:
queen.suit, queen.rank

インスタンスを使用してクラス変数にアクセスすることも合法です。

In [ ]:
queen.suit_names

ただし、クラスを使用すると、それらが属性ではなくクラス変数であることがより明確になります。

もちろん、`Card`オブジェクトの`__str__`メソッドの定義を以下に示します。`Card`クラスは一般的にトランプのカードを表現するために利用されます。簡単な例としてカードの`__str__`メソッドを実装すると、以下のようになります。

```python
class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank

    def __str__(self):
        return f"{self.rank} of {self.suit}"

# 使用例
card = Card("Hearts", "Ace")
print(card)  # 出力: Ace of Hearts
```

ここで、`suit`はカードのスート（例：Hearts, Diamonds, Clubs, Spades）を表し、`rank`はカードの数字や顔（例：2, 3, ... , 10, Jack, Queen, King, Ace）を表します。`__str__`メソッドはカードオブジェクトが文字列として表現されるときの形式を定義します。この例では、「rank of suit」という形式で出力されます。

In [ ]:
%%add_method_to Card

    def __str__(self):
        rank_name = Card.rank_names[self.rank]
        suit_name = Card.suit_names[self.suit]
        return f'{rank_name} of {suit_name}'

`Card`を印刷すると、Pythonは`__str__`メソッドを呼び出してカードの人間に読みやすい表現を取得します。

In [ ]:
print(queen)

以下は`Card`クラスオブジェクトとそのインスタンスである`queen`の図です。
`Card`はクラスオブジェクトなので、その型は`type`です。
`queen`は`Card`のインスタンスなので、その型は`Card`です。
スペースを節約するために、`suit_names`と`rank_names`の中身は描いていません。

In [ ]:
from diagram import Binding, Value, Frame, Stack

bindings = [Binding(Value(name), draw_value=False)
            for name in ['suit_names', 'rank_names']]

frame1 = Frame(bindings, name='type', dy=-0.5, offsetx=0.77)
binding1 = Binding(Value('Card'), frame1)

bindings = [Binding(Value(name), Value(value))
            for name, value in zip(['suit', 'rank'], [1, 11])]

frame2 = Frame(bindings, name='Card', dy=-0.3, offsetx=0.33)
binding2 = Binding(Value('queen'), frame2)

stack = Stack([binding1, binding2], dy=-1.2)

In [ ]:
from diagram import diagram, Bbox, make_list, adjust

width, height, x, y = [2.11, 2.14, 0.35, 1.76]
ax = diagram(width, height)
bbox = stack.draw(ax, x, y)

value = make_list([])
bbox2 = value.draw(ax, x+1.66, y)

value = make_list([])
bbox3 = value.draw(ax, x+1.66, y-0.5)

bbox = Bbox.union([bbox, bbox2, bbox3])
#adjust(x, y, bbox)

各 `Card` インスタンスはそれぞれ独自の `suit` と `rank` 属性を持ちますが、`Card` クラスオブジェクトは1つだけであり、クラス変数 `suit_names` と `rank_names` のコピーも1つだけです。

カードのオブジェクトをもう一つ作成し、同じスートとランクを持たせるとしましょう。

In [ ]:
queen2 = Card(1, 12)
print(queen2)

`==` 演算子を使用してそれらを比較する場合、`queen` と `queen2` が同じオブジェクトを参照しているかどうかを確認します。

In [ ]:
queen == queen2

それに一致しないため、`False`を返します。
この動作は、特別なメソッド`__eq__`を定義することで変更できます。

In [ ]:
%%add_method_to Card

    def __eq__(self, other):
        return self.suit == other.suit and self.rank == other.rank

`__eq__`メソッドは、2つの`Card`オブジェクトをパラメータとして受け取り、それらが同じスートとランクを持っている場合に`True`を返します。つまり、それらが同一のオブジェクトでなくても等価であるかを確認します。

`Card`オブジェクト同士で`==`演算子を使用すると、Pythonは`__eq__`メソッドを呼び出します。

In [ ]:
queen == queen2

2番目のテストとして、同じスートで異なるランクのカードを作成してみましょう。

In [ ]:
six = Card(1, 6)
print(six)

`queen`と`six`が同等ではないことを確認できます。

In [ ]:
queen == six

`!=` 演算子を使用する場合、Python は `__ne__` と呼ばれる特別なメソッドを呼び出します。ただし、このメソッドが存在しない場合は `__eq__` を呼び出してその結果を反転させます。つまり、`__eq__` が `True` を返す場合、`!=` 演算子の結果は `False` になります。

In [ ]:
queen != queen2

In [ ]:
queen != six

2枚のカードを比較してどちらが大きいかを判断したい場合、関係演算子を使用すると`TypeError`が発生することがあります。これは、カードの比較が通常の数値や文字列とは異なる基準で行われるためです。カードオブジェクト同士を直接比較するには、カードのランクやスートに基づいたカスタムの比較メソッドを実装する必要があります。例えば、Pythonでは`__lt__`や`__gt__`といった特殊メソッドを定義することで、カードの比較をコントロールすることが可能です。

In [ ]:
%%expect TypeError

queen < queen2

`<` 演算子の動作を変更するためには、`__lt__` という特別なメソッドを定義できます。これは "less than" の略です。この例では、スートがランクよりも重要であると仮定します。したがって、すべてのスペードはすべてのハートよりも優れており、ハートはすべてのダイヤモンドよりも優れています。もし2枚のカードが同じスートを持つ場合は、ランクが高いカードが勝ちます。

このロジックを実装するために、以下のメソッドを使用します。このメソッドは、カードのスートとランクをその順で含むタプルを返します。

In [ ]:
%%add_method_to Card

    def to_tuple(self):
        return (self.suit, self.rank)

このメソッドを使用して `__lt__` を記述することができます。

In [ ]:
%%add_method_to Card

    def __lt__(self, other):
        return self.to_tuple() < other.to_tuple()

タプルの比較では、まず各タプルの最初の要素、つまりスートを比較します。もしそれらが同じであれば、次の要素、つまりランクを比較します。

`<` 演算子を使用すると、`__lt__` メソッドが呼び出されます。

In [ ]:
six < queen

`>`演算子を使用すると、`__gt__`という特別なメソッドが存在すればそれを呼び出します。
存在しなければ、引数を逆の順番にして`__lt__`を呼び出します。

In [ ]:
queen < queen2

In [ ]:
queen > queen2

最後に、`<=` 演算子を使用すると、`__le__` という特別なメソッドが呼び出されます。

In [ ]:
%%add_method_to Card

    def __le__(self, other):
        return self.to_tuple() <= other.to_tuple()

だから、あるカードが別のカードより小さいか、または等しいかどうかを確認できます。

In [ ]:
queen <= queen2

In [ ]:
queen <= six

`>=` 演算子を使用すると、もし `__ge__` が存在する場合、それが使用されます。存在しない場合は、引数の順序を逆にして `__le__` が呼び出されます。

In [ ]:
queen >= six

これらのメソッドを定義したように、それらは任意の2つの`Card`オブジェクトを比較できるという意味で完全であり、異なる演算子からの結果が互いに矛盾しないという意味で一貫しています。 これらの2つの特性により、`Card`オブジェクトは**完全順序付け**されていると言えます。 これはすぐにわかるように、それらをソートできることを意味します。

## デッキ

カードを表すオブジェクトが定義できたので、デッキを表すオブジェクトを定義しましょう。
以下は、`Deck` クラスの定義です。このクラスには、`Card` オブジェクトのリストをパラメータとして受け取り、それを `cards` という属性に割り当てる `__init__` メソッドがあります。

In [ ]:
class Deck:

    def __init__(self, cards):
        self.cards = cards

標準的なトランプデッキに含まれる52枚のカードのリストを作成するために、次の静的メソッドを使用します。

In [ ]:
%%add_method_to Deck

    def make_cards():
        cards = []
        for suit in range(4):
            for rank in range(2, 15):
                card = Card(suit, rank)
                cards.append(card)
        return cards

`make_cards`内では、外側のループがスートを`0`から`3`まで列挙します。
内側のループではランクを`2`から`14`まで列挙し、`14`はキングよりランクが高いエースを表します。
各イテレーションでは、現在のスートとランクを持つ新しい`Card`を作成し、それを`cards`に追加します。

以下に、カードのリストとそれを含む`Deck`オブジェクトを作成する方法を示します。

In [ ]:
cards = Deck.make_cards()
deck = Deck(cards)
len(deck.cards)

それには、意図通り52枚のカードが含まれています。

デッキクラスの`__str__`メソッドは、デッキの内容を文字列として表現するためのものです。このメソッドを実装することで、デッキオブジェクトをprint関数で出力する際に、カードを視覚的に確認できるようになります。

例として、デッキ内のカードを一行に一枚ずつ表示するような`__str__`メソッドの実装を考えてみましょう。

```python
class Deck:
    def __init__(self, cards):
        self.cards = cards

    def __str__(self):
        return '\n'.join(str(card) for card in self.cards)
```

上記のコードでは、`Deck`クラスにはカードのリストを保持する`cards`という属性があります。`__str__`メソッド内で、このリストの各カードを文字列に変換し、それらを改行区切りで結合して、一つの文字列として返しています。

このように実装すれば、デッキの内容をprint関数で簡単に出力することができ、各カードが見やすく表示されます。

In [ ]:
%%add_method_to Deck

    def __str__(self):
        res = []
        for card in self.cards:
            res.append(str(card))
        return '\n'.join(res)

この方法は、大きな文字列を効率的に蓄積する方法を示しています。まず文字列のリストを作成し、その後に文字列のメソッド `join` を使用します。

この方法を、2枚のカードしか含まないデッキでテストしてみましょう。

In [ ]:
small_deck = Deck([queen, six])

`str`を呼び出すと、`__str__`が呼び出されます。

In [ ]:
str(small_deck)

Jupyterが文字列を表示するとき、「表現形式」を示します。これは改行をシーケンス`\n`で表します。

しかし、結果をprint関数で表示すると、Jupyterは文字列の「印刷可能形式」を示し、改行を空白として表示します。

In [ ]:
print(small_deck)

そのカードは別々の行に表示されます。

カードを配るには、デッキからカードを取り除き、それを返すメソッドが必要です。そのために、リストメソッドの`pop`を使うと便利です。

In [ ]:
%%add_method_to Deck

    def take_card(self):
        return self.cards.pop()

ここでの使用方法です。

In [ ]:
card = deck.take_card()
print(card)

デッキにはカードが「51」枚残っていることを確認できます。

In [ ]:
len(deck.cards)

カードを追加するには、リストメソッド`append`を使用できます。

In [ ]:
%%add_method_to Deck

    def put_card(self, card):
        self.cards.append(card)

例として、今取り出したカードを戻すことができます。

In [ ]:
deck.put_card(card)
len(deck.cards)

デッキをシャッフルするには、`random`モジュールの`shuffle`関数を使用できます。

In [ ]:
import random

In [ ]:
# This cell initializes the random number generator so we
# always get the same results.

random.seed(3)

In [ ]:
%%add_method_to Deck

    def shuffle(self):
        random.shuffle(self.cards)

デッキをシャッフルして最初の数枚を印刷すると、それらがはっきりした順序になっていないことがわかります。

In [ ]:
deck.shuffle()
for card in deck.cards[:4]:
    print(card)

カードを並べ替えるには、リストメソッドの`sort`を使用できます。このメソッドは要素を「インプレース」で並べ替えます。つまり、新しいリストを作成するのではなく、リストを変更します。

In [ ]:
%%add_method_to Deck

    def sort(self):
        self.cards.sort()

`sort`を呼び出すと、カードの比較に`__lt__`メソッドが使用されます。

In [ ]:
deck.sort()

最初の数枚のカードを印刷すれば、それらが昇順になっていることを確認できます。

In [ ]:
for card in deck.cards[:4]:
    print(card)

この例では、`Deck.sort`は`list.sort`を呼び出す以外には何もしていません。
このように責任を引き渡すことを**委譲**と呼びます。

## 親と子

継承とは、既存のクラスを修正したバージョンとして新しいクラスを定義する能力です。
例として、「手札」を表すクラスを作成したいとしましょう。これは、1人のプレイヤーが持っているカードです。

* 手札はデッキに似ています。どちらもカードのコレクションで構成されており、カードの追加や削除といった操作が必要です。

* 手札はデッキと異なります。手札に必要な操作がデッキには意味をなさない場合があります。例えば、ポーカーでは2つの手札を比較してどちらが勝つかを判断することがあります。ブリッジでは、入札を行うために手札のスコアを計算することがあります。

このように、あるクラスが他のクラスの特化したバージョンであるという関係は、継承に適しています。

既存のクラスを基にした新しいクラスを定義するには、既存のクラス名を括弧内に記述します。

In [ ]:
class Hand(Deck):
    """Represents a hand of playing cards."""

この定義は、`Hand` が `Deck` を継承していることを示しています。つまり、`Hand` オブジェクトは `Deck` に定義されている `take_card` や `put_card` といったメソッドにアクセスできます。

また、`Hand` は `Deck` から `__init__` も継承しますが、もし `Hand` クラスで `__init__` を定義すると、それは `Deck` クラスのものをオーバーライドします。

In [ ]:
%%add_method_to Hand

    def __init__(self, label=''):
        self.label = label
        self.cards = []

このバージョンの `__init__` は、オプションの文字列をパラメータとして受け取り、常に空のカードのリストから始まります。`Hand` を作成すると、Python はこのメソッドを呼び出し、`Deck` のメソッドを呼び出さないことになります。それを確認する方法としては、結果として得られるオブジェクトに `label` 属性があるかどうかをチェックすることが挙げられます。

In [ ]:
hand = Hand('player 1')
hand.label

カードを配るために、`take_card` を使用して `Deck` からカードを取り除き、`put_card` を使用してそのカードを `Hand` に追加することができます。

In [ ]:
deck = Deck(cards)
card = deck.take_card()
hand.put_card(card)
print(hand)

このコードを `move_cards` という `Deck` メソッドにカプセル化しましょう。

In [ ]:
%%add_method_to Deck

    def move_cards(self, other, num):
        for i in range(num):
            card = self.take_card()
            other.put_card(card)

このメソッドは多態的です。つまり、`self` と `other` は `Hand` または `Deck` のどちらかの型として機能することができます。そのため、このメソッドを使用して、`Deck` から `Hand` にカードを配る、ある `Hand` から別の `Hand` にカードを渡す、あるいは `Hand` から `Deck` にカードを戻すことができます。

新しいクラスが既存のクラスを継承すると、既存のクラスは「親」と呼ばれ、新しいクラスは「子」と呼ばれます。一般的に：

* 子クラスのインスタンスは、親クラスのすべての属性を持つべきですが、追加の属性を持つことができます。

* 子クラスは、親クラスのすべてのメソッドを持つべきですが、追加のメソッドを持つことができます。

* 子クラスが親クラスのメソッドをオーバーライドする場合、新しいメソッドは同じパラメータを取り、互換性のある結果を返すべきです。

この一連のルールは、計算機科学者バーバラ・リスコフにちなんで「リスコフの置換原則」と呼ばれています。

これらのルールに従えば、「Deck」のような親クラスのインスタンスで動作するように設計された任意の関数やメソッドは、「Hand」のような子クラスのインスタンスでも動作します。これらのルールに違反すると、コードはトランプの家のように崩れてしまいます（すみません）。

ブリッジゲームの手札を表すクラス `BridgeHand` を作成しましょう。このクラスは、カードゲームの手札を表す `Hand` クラスを継承し、高いカードを評価する新しいメソッド `high_card_point_count` を追加します。このメソッドは、手札の高いカードに応じてポイントを加算することで評価を行います。

以下にクラス定義があります。クラス変数としてカードの名前からポイント値へのマッピングを行う辞書も含めます。

```python
class Hand:
    def __init__(self, cards):
        self.cards = cards

class BridgeHand(Hand):
    HIGH_CARD_POINTS = {
        'A': 4,  # Ace
        'K': 3,  # King
        'Q': 2,  # Queen
        'J': 1   # Jack
    }

    def high_card_point_count(self):
        count = 0
        for card in self.cards:
            rank = card[0]  # Assuming card is a string like 'AS' (Ace of Spades)
            if rank in self.HIGH_CARD_POINTS:
                count += self.HIGH_CARD_POINTS[rank]
        return count
```

このコードでは、`BridgeHand` クラスにおける `high_card_point_count` メソッドが、カードのランクに基づいてポイントを計算します。`HIGH_CARD_POINTS` 辞書を用いて、エース、キング、クイーン、ジャックにそれぞれのポイントを割り当てます。

In [ ]:
class BridgeHand(Hand):
    """Represents a bridge hand."""

    hcp_dict = {
        'Ace': 4,
        'King': 3,
        'Queen': 2,
        'Jack': 1,
    }

カードのランク（例えば `12`）が与えられた場合、`Card.rank_names`を使用してそのランクの文字列表現を取得し、次に`hcp_dict`を使用してそのスコアを得ることができます。

In [ ]:
rank = 12
rank_name = Card.rank_names[rank]
score = BridgeHand.hcp_dict.get(rank_name, 0)
rank_name, score

次のメソッドは、`BridgeHand` 内のカードをループして、それらのスコアを合計します。

In [ ]:
%%add_method_to BridgeHand

    def high_card_point_count(self):
        count = 0
        for card in self.cards:
            rank_name = Card.rank_names[card.rank]
            count += BridgeHand.hcp_dict.get(rank_name, 0)
        return count

In [ ]:
# This cell makes a fresh Deck and
# initializes the random number generator

cards = Deck.make_cards()
deck = Deck(cards)
random.seed(3)

テストのために、5枚のカードでハンドを配ります。通常、ブリッジのハンドは13枚ですが、小さい例でコードをテストする方が簡単です。

In [ ]:
hand = BridgeHand('player 2')

deck.shuffle()
deck.move_cards(hand, 5)
print(hand)

こちらが国王と王妃の合計得点です。

In [ ]:
hand.high_card_point_count()

`BridgeHand`は、`Hand`の変数やメソッドを継承し、ブリッジに特化したクラス変数とメソッドを追加します。
継承を利用したこの方法は、特定の用途（例えばブリッジをプレイすること）に特化した新しいクラスを定義するため、**特殊化**と呼ばれます。

## デバッグ

継承は便利な機能です。継承がなければ繰り返しが多くなるプログラムも、継承を使うことで簡潔に書くことができます。また、継承を利用することで親クラスを修正することなく、その動作をカスタマイズできるため、コードの再利用が可能になります。場合によっては、継承の構造が問題の自然な構造を反映し、設計が理解しやすくなることもあります。

一方で、継承はプログラムの可読性を下げることがあります。メソッドが呼び出されたときに、その定義がどこにあるのか分かりにくい場合があり、関連するコードが複数のモジュールに分散していることがあります。

プログラムの実行フローに確信が持てない場合、最も簡単な解決策は関連するメソッドの冒頭にprint文を追加することです。例えば、`Deck.shuffle`が`Running Deck.shuffle`のようなメッセージを出力するようにすれば、プログラムの実行中にそのフローを追跡することができます。

別の方法として、以下の関数を使用することができます。この関数はオブジェクトとメソッド名（文字列として）を受け取り、そのメソッドを定義しているクラスを返します。

In [ ]:
def find_defining_class(obj, method_name):
    """Find the class where the given method is defined."""
    for typ in type(obj).mro():
        if method_name in vars(typ):
            return typ
    return f'Method {method_name} not found.'

`find_defining_class`は、メソッドを探すクラスオブジェクト（型）のリストを取得するために`mro`メソッドを使用します。「MRO」は「メソッド解決順序」を意味し、Pythonがメソッド名を「解決」するために検索するクラスのシーケンスです。これは、メソッド名が参照する関数オブジェクトを見つけるためのものです。

例として、`BridgeHand`をインスタンス化し、その後`shuffle`メソッドの定義クラスを見つけましょう。

In [ ]:
hand = BridgeHand('player 3')
find_defining_class(hand, 'shuffle')

`BridgeHand`オブジェクトの`shuffle`メソッドは`Deck`のものです。

## 用語集

**継承 (いしょう):**
 以前に定義されたクラスを修正したバージョンとして、新しいクラスを定義する能力。

**エンコード (えんこーど):**
 一組の値を、他の一組の値を使用して表現するために、それらの間にマッピングを構築すること。

**クラス変数 (くらすへんすう):**
 クラス定義内で定義されているが、いずれのメソッド内にもない変数。

**全順序 (ぜんじゅんじょ):**
 オブジェクトの集合が全順序であるとは、任意の二要素を比較し、その結果が一貫している場合を指す。

**委譲 (いじょう):**
 あるメソッドが、責任を別のメソッドに渡し、ほとんどまたはすべての作業をそのメソッドに任せること。

**親クラス (おやくらす):**
 継承されるクラス。

**子クラス (こくらす):**
 他のクラスから継承するクラス。

**特化 (とっか):**
 継承を利用して、既存のクラスの特化バージョンとして新しいクラスを作成する方法。

## 練習問題

In [ ]:
# This cell tells Jupyter to provide detailed debugging information
# when a runtime error occurs. Run it before working on the exercises.

%xmode Verbose

### 仮想アシスタントに質問

オブジェクト指向プログラミング（OOP）がうまくいくと、プログラムがより読みやすく、テストしやすく、再利用しやすくなります。しかし、プログラムが複雑で維持しにくくなる可能性もあります。その結果、OOPは議論の的となっており、愛する人もいれば、そうでない人もいます。

このトピックについてもっと知るために、仮想アシスタントに以下を尋ねましょう。

* オブジェクト指向プログラミングの長所と短所は何ですか？

* 「継承よりもコンポジションを優先する」とはどういう意味ですか？

* リスコフの置換原則とは何ですか？

* Pythonはオブジェクト指向言語ですか？

* 集合が全順序であるための条件は何ですか？

また、これらのエクササイズを行う際には、仮想アシスタントを活用することを検討してください。

### 演習

コントラクトブリッジにおいて、「トリック」とは、4人のプレイヤーがそれぞれ1枚のカードをプレイするラウンドのことを指します。
これらのカードを表現するために、`Deck`クラスを継承したクラスを定義します。

In [ ]:
class Trick(Deck):
    """Represents a trick in contract bridge."""

例えばこのトリックを考えてみましょう。最初のプレイヤーがダイヤの3を出すと、ダイヤが「リードスート」となります。2番目と3番目のプレイヤーは「フォロースート」し、リードスートのカードを出します。4番目のプレイヤーは別のスートのカードを出すため、このトリックには勝てません。したがって、このトリックの勝者は3番目のプレイヤーで、リードスートの中で最も高いカードを出したからです。

In [ ]:
cards = [Card(1, 3),
         Card(1, 10),
         Card(1, 12),
         Card(2, 13)]
trick = Trick(cards)
print(trick)

トランプゲームにおける`Trick`クラスにおいて、トリック内のカードを走査し、勝利するカードのインデックスを返す`find_winner`メソッドを実装する方法を説明します。以下のコードスニペットでは、その方法を示します。

```python
class Trick:
    def __init__(self, cards):
        # カードはリストとして与えられると仮定します
        self.cards = cards

    def find_winner(self):
        # ここでは、勝利するカードのインデックスを見つける方法を示します
        winning_card_index = 0
        winning_card = self.cards[0]
        
        for i in range(1, len(self.cards)):
            current_card = self.cards[i]
            
            # 比較ロジック: ここで具体的な比較ルールを実装します
            # 例として、単純に値が大きいカードを勝ちとする場合
            if current_card > winning_card:
                winning_card = current_card
                winning_card_index = i

        return winning_card_index

# 使用例
cards = [3, 6, 10, 7]  # 例としてのカードリスト
trick = Trick(cards)
winner_index = trick.find_winner()
print(f"The index of the winning card is: {winner_index}")  # 期待される出力は2
```

このコードは、カードリストの中で最も高い値を持つカードのインデックスを`find_winner`メソッドで返します。この例では、単純な整数としてカードを比較していますが、実際のトランプゲームでは、カードのスートや特別なルールに基づいて比較を行う必要があるかもしれません。その場合は、比較ロジック部分を適切に調整してください。

以下のアウトラインを使用して始めてください。

In [ ]:
%%add_method_to Trick

    def find_winner(self):
        return 0

In [ ]:
# Solution goes here

以前の例でメソッドをテストする場合、勝利カードのインデックスは「2」であるべきです。

In [ ]:
trick.find_winner()

### 演習

次のいくつかの演習では、ポーカーの手札を分類する関数を書くことが求められています。
ポーカーに詳しくない場合、必要な情報を説明します。
以下のクラスを使用してポーカーの手札を表現します。

In [ ]:
class PokerHand(Hand):
    """Represents a poker hand."""

    def get_suit_counts(self):
        counter = {}
        for card in self.cards:
            key = card.suit
            counter[key] = counter.get(key, 0) + 1
        return counter

    def get_rank_counts(self):
        counter = {}
        for card in self.cards:
            key = card.rank
            counter[key] = counter.get(key, 0) + 1
        return counter

`PokerHand` は、いくつかの演習を助けるための2つのメソッドを提供しています。

* `get_suit_counts` は `PokerHand` の中のカードをループで回り、各スーツのカードの数を数え、それをスーツコードから出現回数へのマッピングを持つ辞書として返します。

* `get_rank_counts` はカードのランクについて同様の操作を行い、ランクコードから出現回数へのマッピングを持つ辞書として返します。

以下の演習はすべて、これまでに学んだPythonの機能のみで実行できますが、そのうちのいくつかはこれまでの演習より難しいです。
仮想アシスタントに助けを求めることをお勧めします。

このような問題では、戦略やアルゴリズムについて一般的なアドバイスを求めると良い結果が得られることが多いです。
その後、自分でコードを書くか、コードを求めることができます。
もしコードを依頼する場合は、関連するクラス定義をプロンプトの一部として提供することを検討してください。

最初の練習として、手札に「フラッシュ」があるかどうかを確認するメソッド`has_flush`を作成します。ここでいう「フラッシュ」とは、同じスートのカードが少なくとも5枚含まれていることを意味します。

ほとんどのポーカーの種類では、手札には5枚または7枚のカードが含まれますが、手札に他の枚数のカードが含まれるエキゾチックなバリエーションも存在します。しかし、手札に含まれるカードが何枚であっても、最良のハンドを作る5枚のカードのみがカウントされます。

以下のアウトラインを使って始めることができます。

In [ ]:
%%add_method_to PokerHand

    def has_flush(self):
        """Checks whether this hand has a flush."""
        return False

In [ ]:
# Solution goes here

この方法をテストするために、すべてがクラブの5枚のカードを持つ手札を構築し、フラッシュを含むようにします。

In [ ]:
good_hand = PokerHand('good_hand')

suit = 0
for rank in range(10, 15):
    card = Card(suit, rank)
    good_hand.put_card(card)

print(good_hand)

`get_suit_counts`を呼び出すと、ランクコード`0`が`5`回出現することを確認できます。

In [ ]:
good_hand.get_suit_counts()

したがって、`has_flush` は `True` を返すべきです。

In [ ]:
good_hand.has_flush()

2回目のテストとして、3枚のクラブと他の2種類のスーツで手札を構成します。

In [ ]:
cards = [Card(0, 2),
         Card(0, 3),
         Card(2, 4),
         Card(3, 5),
         Card(0, 7),
        ]

bad_hand = PokerHand('bad hand')
for card in cards:
    bad_hand.put_card(card)

print(bad_hand)

したがって、`has_flush`は`False`を返すべきです。

In [ ]:
bad_hand.has_flush()

### エクササイズ

ハンドにストレート（連続するランクを持つ5枚のカードのセット）が含まれているかどうかを確認する`has_straight`メソッドを書いてください。
例えば、ハンドにランク`5`、`6`、`7`、`8`、`9`が含まれている場合、それはストレートを含んでいます。

エースは2の前、またはキングの後に来ることができるため、`Ace`、`2`、`3`、`4`、`5`はストレートであり、`10`、`Jack`、`Queen`、`King`、`Ace`もストレートです。
しかし、ストレートは「ラップアラウンド」することはできないため、`King`、`Ace`、`2`、`3`、`4`はストレートではありません。

以下のアウトラインを使用して開始できます。これには、エースの数を数えるいくつかのコード行が含まれており、エースはコード `1` または `14` で表され、カウンターの両方の位置に合計が格納されます。

In [ ]:
%%add_method_to PokerHand

    def has_straight(self, n=5):
        """Checks whether this hand has a straight with at least `n` cards."""
        counter = self.get_rank_counts()
        aces = counter.get(1, 0) + counter.get(14, 0)
        counter[1] = aces
        counter[14] = aces

        return False

In [ ]:
# Solution goes here

前回の演習で作成した「good_hand」にはストレートが含まれています。「get_rank_counts」を使用すると、5つの連続したランクの各ランクに少なくとも1枚のカードがあることを確認できます。

In [ ]:
good_hand.get_rank_counts()

したがって、`has_straight`は`True`を返すべきです。

In [ ]:
good_hand.has_straight()

`bad_hand`にはストレートが含まれていないため、`has_straight`は`False`を返すべきです。

In [ ]:
bad_hand.has_straight()

ポーカーの手札にストレートフラッシュが含まれているかどうかを確認するための方法を設計します。ストレートフラッシュは、5枚の同じスーツかつ連続したランクのカードで構成されます。以下に、Pythonで`PokerHand`クラスのメソッドを実装する方法を示します。

```python
class Card:
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

class PokerHand:
    def __init__(self, cards):
        self.cards = cards

    def has_straight_flush(self):
        # カードをスーツごとに分類
        suits = {}
        for card in self.cards:
            if card.suit not in suits:
                suits[card.suit] = []
            suits[card.suit].append(card.rank)

        # 各スーツに対して、ストレートが存在するか確認
        for suit, ranks in suits.items():
            # ランクを昇順にソート（エースを考慮して13以下の範囲に対応）
            ranks = list(set(ranks))  # 重複があれば削除
            ranks.sort()
            
            # ストレートチェック（エースは14として扱う可能性があるため、追加）
            rank_values = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, 'J': 11, 'Q': 12, 'K': 13, 'A': 14}
            rank_indices = [rank_values[rank] for rank in ranks]
            
            # ストレートの5連続を確認
            for i in range(len(rank_indices) - 4):
                if rank_indices[i + 4] == rank_indices[i] + 4:
                    return True

        return False
```

この実装では、まず手札のカードをスーツごとに分類し、各スーツについてストレートが存在するかチェックします。これを行うことで、ストレートフラッシュの存在を確認します。

以下のアウトラインを使って始めてください。

In [ ]:
%%add_method_to PokerHand

    def has_straightflush(self):
        """Check whether this hand has a straight flush."""
        return False

In [ ]:
# Solution goes here

In [ ]:
# Solution goes here

メソッドをテストするための以下の例を使用してください。

In [ ]:
good_hand.has_straightflush()     # should return True

In [ ]:
bad_hand.has_straightflush()     # should return False

ストレートとフラッシュがあるかどうかを確認するだけでは不十分であることに注意してください。その理由を考えるために、次の手札を考えてみましょう。

In [ ]:
from copy import deepcopy

straight_and_flush = deepcopy(bad_hand)
straight_and_flush.put_card(Card(0, 6))
straight_and_flush.put_card(Card(0, 9))
print(straight_and_flush)

この手にはストレートとフラッシュが含まれていますが、それらは同じ5枚のカードではありません。

In [ ]:
 straight_and_flush.has_straight(), straight_and_flush.has_flush()

したがって、ストレートフラッシュは含まれていません。

In [ ]:
straight_and_flush.has_straightflush()    # should return False

```python
class PokerHand:
    def __init__(self, cards):
        self.cards = cards

    def has_pair(self):
        rank_count = {}
        for card in self.cards:
            rank = card[:-1]  # トランプランクを取得（例: '9' in '9H'）
            if rank in rank_count:
                rank_count[rank] += 1
            else:
                rank_count[rank] = 1
        
        for count in rank_count.values():
            if count >= 2:
                return True
        return False

# 使用例
hand = PokerHand(['9H', '9D', '5S', '3C', '2H'])
print(hand.has_pair())  # Trueを出力
```

このコードは、`PokerHand` クラスに `has_pair` メソッドを追加し、ポーカーハンドにペアが存在するかどうかをチェックします。カードのランクごとにカウントを行い、2枚以上のカードが同じランクであればペアがあると判断します。

次のアウトラインを使用して開始できます。

In [ ]:
%%add_method_to PokerHand

    def check_sets(self, *need_list):
        return True

In [ ]:
# Solution goes here

In [ ]:
# Solution goes here

テストするために、ペアがある手札を紹介します。

In [ ]:
pair = deepcopy(bad_hand)
pair.put_card(Card(1, 2))
print(pair)

In [ ]:
pair.has_pair()    # should return True

In [ ]:
bad_hand.has_pair()    # should return False

In [ ]:
good_hand.has_pair()   # should return False

ポーカーハンドのフルハウスを判定するメソッドを実装するためには、手札のカードをランクごとに分類し、その出現回数を確認する必要があります。以下にそのメソッドのサンプル実装を示します：

```python
class PokerHand:
    def __init__(self, cards):
        self.cards = cards

    def has_full_house(self):
        rank_counts = {}

        # ランクごとのカウントを取得
        for card in self.cards:
            rank = card['rank']
            if rank in rank_counts:
                rank_counts[rank] += 1
            else:
                rank_counts[rank] = 1

        # 出現回数をリストに変換
        counts = list(rank_counts.values())

        # フルハウスは3枚が1つと2枚が1つ
        return (3 in counts) and (2 in counts)
```

このメソッドでは、`cards`はカードのリストであり、それぞれのカードはランク情報を含む辞書型オブジェクトと仮定しています。`has_full_house`メソッドは手札がフルハウスかどうかを確認し、フルハウスであれば`True`を、そうでなければ`False`を返します。

次のアウトラインを使用して始めることができます。

In [ ]:
%%add_method_to PokerHand

    def has_full_house(self):
        return False

In [ ]:
# Solution goes here

この手を使ってあなたの方法をテストできます。

In [ ]:
boat = deepcopy(pair)
boat.put_card(Card(2, 2))
boat.put_card(Card(2, 3))
print(boat)

In [ ]:
boat.has_full_house()     # should return True

In [ ]:
pair.has_full_house()     # should return False

In [ ]:
good_hand.has_full_house()     # should return False

### 演習

この演習は、デバッグが難しい一般的なエラーについての警告の物語です。次のクラス定義を考えてみましょう。

In [ ]:
class Kangaroo:
    """A Kangaroo is a marsupial."""

    def __init__(self, name, contents=[]):
        """Initialize the pouch contents.

        name: string
        contents: initial pouch contents.
        """
        self.name = name
        self.contents = contents

    def __str__(self):
        """Return a string representaion of this Kangaroo.
        """
        t = [ self.name + ' has pouch contents:' ]
        for obj in self.contents:
            s = '    ' + object.__str__(obj)
            t.append(s)
        return '\n'.join(t)

    def put_in_pouch(self, item):
        """Adds a new item to the pouch contents.

        item: object to be added
        """
        self.contents.append(item)

`__init__`は2つのパラメータを受け取ります。`name`は必須ですが、`contents`はオプションです。`contents`が提供されない場合は、デフォルト値として空のリストが使用されます。

`__str__`は、オブジェクトの文字列表現を返します。この文字列表現には、ポーチの名前と内容が含まれます。

`put_in_pouch`は任意のオブジェクトを受け取り、それを`contents`に追加します。

では、このクラスがどのように動作するかを見てみましょう。
`Kangaroo`オブジェクトを名前`'Kanga'`と`'Roo'`で2つ作成します。

In [ ]:
kanga = Kangaroo('Kanga')
roo = Kangaroo('Roo')

カンガのポーチには、2本の紐とルーを加えます。

In [ ]:
kanga.put_in_pouch('wallet')
kanga.put_in_pouch('car keys')
kanga.put_in_pouch(roo)

「kanga」を印刷すれば、すべてがうまくいったように思えます。

In [ ]:
print(kanga)

しかし、`roo`を印刷するとどうなりますか？

In [ ]:
print(roo)

ルーのポーチには、カンガのと同じ内容物が含まれています。その中には、`roo`の参照もあります。

何が問題だったのかを考えてみてください。
その後、バーチャルアシスタントに「次のプログラムの何が問題ですか？」と尋ね、`Kangaroo`の定義を貼り付けてください。

[Think Python: 第3版](https://allendowney.github.io/ThinkPython/index.html)

著作権 2024 [Allen B. Downey](https://allendowney.com)

コードライセンス: [MITライセンス](https://mit-license.org/)

テキストライセンス: [クリエイティブ・コモンズ 表示 - 非営利 - 継承 4.0 国際](https://creativecommons.org/licenses/by-nc-sa/4.0/)